## (1) Load model

In [1]:
from model import Mamba, ModelArgs
from transformers import AutoTokenizer

# One of:
#     'state-spaces/mamba-2.8b-slimpj'
#     'state-spaces/mamba-2.8b'
#     'state-spaces/mamba-1.4b'
#     'state-spaces/mamba-790m'
#     'state-spaces/mamba-370m'
#     'state-spaces/mamba-130m'
pretrained_model_name_list = ["state-spaces/mamba-2.8b-slimpj", "state-spaces/mamba-2.8b", "state-spaces/mamba-1.4b", "state-spaces/mamba-790m", "state-spaces/mamba-370m", "state-spaces/mamba-130m"]
pretrained_model_name = pretrained_model_name_list[-1]
print("The pretrained model I'm using is:", pretrained_model_name)

model = Mamba.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
The pretrained model I'm using is: state-spaces/mamba-130m


/home/aiops/wangsd/miniforge3/envs/env_mamba/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## (2) Generate Text

In [2]:
import torch
import torch.nn.functional as F


def generate(model,
             tokenizer,
             prompt: str,
             n_tokens_to_gen: int = 50,
             sample: bool = True,
             top_k: int = 40):
    model.eval()
    
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    
    for token_n in range(n_tokens_to_gen):
        with torch.no_grad():
            indices_to_input = input_ids
            next_token_logits = model(indices_to_input)[:, -1]
        
        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape
        
        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)
        
        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]
        
        input_ids = torch.cat([input_ids, next_indices], dim=1)

    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]
    
    return output_completions

In [3]:
print(generate(model, tokenizer, 'Mamba is the'))

Mamba is the most important component of C57/B6N background, which we have constructed with two different lines ([@B33]). Two lines, one carrying *hml* and another carrying *hmp* were crossed to identify the null alleles of *h


In [4]:
print(generate(model, tokenizer, 'John: Hi!\nSally:'))

John: Hi!
Sally: Hey!
Mary: Thank you!
John: And thank you so many other folks for coming by to join their teams.
It's my pleasure to welcome you in.
We've got quite a team here, we've had loads



In [5]:
print(generate(model, tokenizer, 'The meaning of life is '))

The meaning of life is 
that it is a combination of many activities
(which include exercise, movement, speech, education, love)
And each life is shaped by it.
This isn't enough, the most important thing is
to have a deep connection with


In [6]:
print(generate(model, tokenizer, 'def reverse_string('))

def reverse_string(str, index):
    '''A function to translate a string into a binary representation
        of a string.  It performs binary search on the
        corresponding sequence.

    >>> reverse_string(['a-z', 'a', '
